<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initializations" data-toc-modified-id="Initializations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initializations</a></span></li><li><span><a href="#General-Questions-and-Comments" data-toc-modified-id="General-Questions-and-Comments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>General Questions and Comments</a></span></li><li><span><a href="#Exact-Solution-for-Convection-Diffusion-Equation" data-toc-modified-id="Exact-Solution-for-Convection-Diffusion-Equation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exact Solution for Convection-Diffusion Equation</a></span></li><li><span><a href="#Discontinuous-Galerkin-for-Convection-Diffusion-with-upwind" data-toc-modified-id="Discontinuous-Galerkin-for-Convection-Diffusion-with-upwind-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Discontinuous Galerkin for Convection-Diffusion with upwind</a></span></li><li><span><a href="#Hybrid-Discontinuous-Galerkin-Convection-Diffusion-with-upwind" data-toc-modified-id="Hybrid-Discontinuous-Galerkin-Convection-Diffusion-with-upwind-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Hybrid Discontinuous Galerkin Convection-Diffusion with upwind</a></span></li><li><span><a href="#Solving-Mass-Matrix-problem" data-toc-modified-id="Solving-Mass-Matrix-problem-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Solving Mass Matrix problem</a></span></li><li><span><a href="#Enrichment" data-toc-modified-id="Enrichment-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Enrichment</a></span><ul class="toc-item"><li><span><a href="#Enrichment-for-Mass-Matrix" data-toc-modified-id="Enrichment-for-Mass-Matrix-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Enrichment for Mass Matrix</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-x-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-x-axis)-7.1.0.1"><span class="toc-item-num">7.1.0.1&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich x-axis)</a></span></li><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-y-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-y-axis)-7.1.0.2"><span class="toc-item-num">7.1.0.2&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich y-axis)</a></span></li><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-both-xy-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-both-xy-axis)-7.1.0.3"><span class="toc-item-num">7.1.0.3&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich both xy-axis)</a></span></li></ul></li></ul></li><li><span><a href="#Enrichment-with-DG" data-toc-modified-id="Enrichment-with-DG-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Enrichment with DG</a></span></li><li><span><a href="#Enrichment-with-HDG" data-toc-modified-id="Enrichment-with-HDG-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Enrichment with HDG</a></span></li></ul></li></ul></div>

In [5]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd

### Initializations

In [6]:
#epsilon = 0.01
# h = uniformly refined mesh
# order = k

In [7]:
## Parameter setup
orders = [1, 2, 3]
beta = (2,0.001)
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
eps = 0.01

# Exact Solution
p = lambda x: x + (exp(beta[0]*x/eps)-1)/(1-exp(beta[0]/eps))
q = lambda y: y + (exp(beta[1]*y/eps)-1)/(1-exp(beta[1]/eps))

p = lambda x: x + (exp(beta[0]*(x-1)/eps)-exp(-beta[0]/eps))/(exp(-beta[0]/eps)-1)
q = lambda y: y + (exp(beta[1]*(y-1)/eps)-exp(-beta[1]/eps))/(exp(-beta[1]/eps)-1)

exact = p(x) * q(y)

#r = lambda x: beta[1] * (1 + ((exp(beta[0] * x / eps) - 1) / (1-exp(beta[0]/eps))))
#s = lambda y: beta[0] * (y + ((exp(beta[1] * y / eps) - 1)/  (1-exp(1/eps))))

coeff =  beta[1] * p(x) +  beta[0] * q(y)
#coeff = r(x) + s(y)

coeff_x = p(x)
coeff_y = q(y)

alpha = 10

## Dataframes
columns = ['Order', 'Mesh Size', 'Error']

### General Questions and Comments

In [4]:
# 1. Check the effect of stabilization paramter on enrichment

### Exact Solution for Convection-Diffusion Equation

In [24]:
orders = 1
mesh_size = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=mesh_size))
fes = L2(mesh, order=orders, dirichlet="bottom|right|left|top")
gfu = GridFunction(fes)
Draw(exact, mesh, 'exact')

### Discontinuous Galerkin for Convection-Diffusion with upwind

In [43]:
dg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dgjumps=True)
        u, v = fes.TnT()

        jump_u = u-u.Other()
        jump_v = v-v.Other()
        n = specialcf.normal(2)
        mean_dudn = 0.5 * n * (grad(u) + grad(u.Other()))
        mean_dvdn = 0.5 * n * (grad(v) + grad(v.Other()))

        h = specialcf.mesh_size

        # diffusion
        diffusion = grad(u) * grad(v) * dx \
            +alpha * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
            +(-mean_dudn * jump_v - mean_dvdn * jump_u) * dx(skeleton=True) \
            +alpha * order ** 2/h * u * v * ds(skeleton=True) \
            + (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

        # convection
        b = CoefficientFunction((beta[0],beta[1]) )
        uup = IfPos(b * n, u, u.Other())
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)
        
        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        acd.Assemble()
        
        # rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        gfu = GridFunction(fes, name="uDG")        
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        #Draw (gfu)
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        dg.loc[len(dg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)
    print('......................................................................')
    
#Draw(gfu,mesh,"gfu")
#Draw(exact, mesh, 'exact')
%store dg

Order: 1 Mesh Size: 1.0 L2-error: 1298.0128018418447
Order: 1 Mesh Size: 0.5 L2-error: 46.5608127161854
Order: 1 Mesh Size: 0.25 L2-error: 13.999999278823037
Order: 1 Mesh Size: 0.125 L2-error: 2.5101102569591958
Order: 1 Mesh Size: 0.0625 L2-error: 0.6527550605845899
Order: 1 Mesh Size: 0.0313 L2-error: 0.1637391633742325
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 20.466258996195986
Order: 2 Mesh Size: 0.5 L2-error: 3.991275051944804
Order: 2 Mesh Size: 0.25 L2-error: 0.9747466059016262
Order: 2 Mesh Size: 0.125 L2-error: 0.24330295663756063
Order: 2 Mesh Size: 0.0625 L2-error: 0.040969352341895145
Order: 2 Mesh Size: 0.0313 L2-error: 0.00760010955031543
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 16.924449384426595
Order: 3 Mesh Size: 0.5 L2-error: 2.3429261316569088
Order: 3 Mesh Size: 0.25 L2-error: 0.37165190488189237
Order: 3 Mesh Size: 0.125 L2-error: 0.0484

### Hybrid Discontinuous Galerkin Convection-Diffusion with upwind

In [5]:
hdg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order)
        F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")

        fes = FESpace([V,F])
        u,uhat = fes.TrialFunction()
        v,vhat = fes.TestFunction()

        jump_u = u-uhat
        jump_v = v-vhat

        condense = True
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)

        dS = dx(element_boundary=True)

        # diffusion
        diffusion = grad(u) * grad(v) * dx + \
            alpha * order ** 2/h * jump_u * jump_v * dS + \
            (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

        # convection            
        b = CoefficientFunction((beta[0],beta[1])) 
        uup = IfPos(b * n, u, uhat)
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

        acd = BilinearForm(fes, condense=condense)
        acd += eps * diffusion + convection
        acd.Assemble()
        
        #rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        gfu = GridFunction(fes)    
        if not condense:
            inv = acd.mat.Inverse(fes.FreeDofs(), "umfpack")
            gfu.vec.data = inv * f.vec
        else:
            f.vec.data += acd.harmonic_extension_trans * f.vec

            inv = acd.mat.Inverse(fes.FreeDofs(True), "umfpack")
            gfu.vec.data = inv * f.vec

            gfu.vec.data += acd.harmonic_extension * gfu.vec
            gfu.vec.data += acd.inner_solve * f.vec
            
        error = sqrt (Integrate ((gfu.components[0] - exact)*(gfu.components[0]- exact), mesh))
        hdg.loc[len(hdg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)                            
    print('......................................................................')

#Draw(gfu.components[0],mesh,"gfu")
%store hdg

Order: 1 Mesh Size: 1.0 L2-error: 0.002138869284735491
Order: 1 Mesh Size: 0.5 L2-error: 0.001544820731014375
Order: 1 Mesh Size: 0.25 L2-error: 0.00030363572952047524
Order: 1 Mesh Size: 0.125 L2-error: 0.0002666348150877068
Order: 1 Mesh Size: 0.0625 L2-error: 0.00025704860358808393
Order: 1 Mesh Size: 0.0313 L2-error: 0.00017363794472098257
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.0009453965867178732
Order: 2 Mesh Size: 0.5 L2-error: 0.0006825357682761736
Order: 2 Mesh Size: 0.25 L2-error: 0.0005912242149777977
Order: 2 Mesh Size: 0.125 L2-error: 0.00040003681554222576
Order: 2 Mesh Size: 0.0625 L2-error: 0.00022951699433768007
Order: 2 Mesh Size: 0.0313 L2-error: 9.068489836841933e-05
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 0.0008425809840232748
Order: 3 Mesh Size: 0.5 L2-error: 0.0008665829826092465
Order: 3 Mesh Size: 0.25 L2-error: 0.0007065485383291

### Solving Mass Matrix problem

In [6]:
mm = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        u, v = fes.TnT()
             
        m = BilinearForm(fes, symmetric=False)
        m += u * v * dx
        m.Assemble()
        
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v)
        f.Assemble()
        
        invmstar = m.mat.Inverse(inverse="umfpack")
        
        gfu = GridFunction(fes)        
        gfu.vec.data = invmstar * f.vec
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        mm.loc[len(mm)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)            
    print('......................................................................')

%store mm

Order: 1 Mesh Size: 1.0 L2-error: 0.004459175075879127
Order: 1 Mesh Size: 0.5 L2-error: 0.0027608054936085454
Order: 1 Mesh Size: 0.25 L2-error: 0.0015883783701244596
Order: 1 Mesh Size: 0.125 L2-error: 0.0009414912851175837
Order: 1 Mesh Size: 0.0625 L2-error: 0.0005427745153344452
Order: 1 Mesh Size: 0.0313 L2-error: 0.00027198692504778744
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.0005208290591023745
Order: 2 Mesh Size: 0.5 L2-error: 0.00011315394599986821
Order: 2 Mesh Size: 0.25 L2-error: 0.00011656823196480515
Order: 2 Mesh Size: 0.125 L2-error: 0.00018361083577536936
Order: 2 Mesh Size: 0.0625 L2-error: 0.00013795691394938493
Order: 2 Mesh Size: 0.0313 L2-error: 5.8390318636803664e-05
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 1.4432766278762056e-06
Order: 3 Mesh Size: 0.5 L2-error: 1.605358539430508e-06
Order: 3 Mesh Size: 0.25 L2-error: 1.6876092453777

### Enrichment

#### Enrichment for Mass Matrix

###### Enrichment for Mass Matrix (enrich x-axis)

In [7]:
emm_x = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        fes = FESpace([V, Qx])
        (us, px), (vs, qx) = fes.TnT()
        
        p = coeff_x * px
        q = coeff_x * qx

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_x.loc[len(emm_x)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    

%store emm_x

Order: 1  Mesh Size: 1.0 L2 Error: 0.002952636476458697
Order: 1  Mesh Size: 0.5 L2 Error: 0.0009501798785003782
Order: 1  Mesh Size: 0.25 L2 Error: 0.00044122188284700687
Order: 1  Mesh Size: 0.125 L2 Error: 0.00016543106730313259
Order: 1  Mesh Size: 0.0625 L2 Error: 0.00016200528488799918
Order: 1  Mesh Size: 0.0313 L2 Error: 6.78015752451131e-05
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0011181913948061298
Order: 2  Mesh Size: 0.5 L2 Error: 0.0002697525546545678
Order: 2  Mesh Size: 0.25 L2 Error: 5.290443957945184e-05
Order: 2  Mesh Size: 0.125 L2 Error: 2.7042155322600637e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.1583824926424019e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 2.2137117459602667e-06
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0005307327802862122
Order: 3  Mesh Size: 0.5 L2 Error: 7.076022323404889e-05
Order: 3  Mesh Size: 0.25 L2 Error: 6.244231763932287e-05
Or

###### Enrichment for Mass Matrix (enrich y-axis)

In [8]:
emm_y = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qy = Compress(Q, active_dofs=ba_y)
        fes = FESpace([V, Qy])
        (us, py), (vs, qy) = fes.TnT()
        
        p = coeff_y * py
        q = coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_y.loc[len(emm_y)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    
%store emm_y

Order: 1  Mesh Size: 1.0 L2 Error: 0.000876897791588433
Order: 1  Mesh Size: 0.5 L2 Error: 0.00028337980147632953
Order: 1  Mesh Size: 0.25 L2 Error: 0.00014865125578281073
Order: 1  Mesh Size: 0.125 L2 Error: 0.00022807369810113462
Order: 1  Mesh Size: 0.0625 L2 Error: 0.0002302250299384858
Order: 1  Mesh Size: 0.0313 L2 Error: 0.00015178222603457261
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0003436446994243978
Order: 2  Mesh Size: 0.5 L2 Error: 0.00012668855441490222
Order: 2  Mesh Size: 0.25 L2 Error: 0.0001373135205300664
Order: 2  Mesh Size: 0.125 L2 Error: 0.0001193670059542598
Order: 2  Mesh Size: 0.0625 L2 Error: 0.00010535130454488087
Order: 2  Mesh Size: 0.0313 L2 Error: 5.160249512132803e-05
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 2.0672900058213586e-05
Order: 3  Mesh Size: 0.5 L2 Error: 0.00017800250410952915
Order: 3  Mesh Size: 0.25 L2 Error: 0.00021788597779965
O

###### Enrichment for Mass Matrix (enrich both xy-axis)

In [9]:
emm_xy = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy])

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = coeff_x * px + coeff_y * py
        q = coeff_x * qx + coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        #Draw(u2-exact,mesh, 'error')
        #Draw(gfu.components[0], mesh, 'testa')
        #Draw(gfu.components[1], mesh, 'testb')
        #Draw(gfu.components[1] * coeff, mesh, 'testc')
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_xy.loc[len(emm_xy)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    
Draw(u2, mesh, 'test')
%store emm_xy

Order: 1  Mesh Size: 1.0 L2 Error: 0.0024882388707216075
Order: 1  Mesh Size: 0.5 L2 Error: 0.0005095316395124877
Order: 1  Mesh Size: 0.25 L2 Error: 0.00044031953449784003
Order: 1  Mesh Size: 0.125 L2 Error: 0.00016543634046145935
Order: 1  Mesh Size: 0.0625 L2 Error: 0.00016196037374068578
Order: 1  Mesh Size: 0.0313 L2 Error: 6.77922579673845e-05
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0010660990044349954
Order: 2  Mesh Size: 0.5 L2 Error: 0.00027173411064011234
Order: 2  Mesh Size: 0.25 L2 Error: 5.3372835205931315e-05
Order: 2  Mesh Size: 0.125 L2 Error: 2.703212300268931e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.1534450351319403e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 2.1883639992342305e-06
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0005350770893206925
Order: 3  Mesh Size: 0.5 L2 Error: 7.117465898487584e-05
Order: 3  Mesh Size: 0.25 L2 Error: 6.282501751556461e-05


In [10]:
## Observations
# Emriching the y axis has a better convergence rate then enriching both x and y
# umfpack fails for this problem

#### Enrichment with DG

In [11]:
## Questions to Explore (TODO)
## (a) Why is the error for mesh size 0.5 larger in this enrichment (val = 20, alpha = 100, eps = 0.01)?
## (b) why is the error for mesh size  0.5 'normal' with the "sparsecholesky" than with "umfpack" (parameters as in (a))?
## (c) What role does the bonusintorder play???
## (d) Changing the bonusintorder still has a bigger error on mesh size 05
## (e) How to check automatically which bonusintorder gives the best convergence rate.

#Observations
### Best Convergence for val=15, umfpack (smallest error rate so far)
### increasing the bousintorder doesn't 'necessarily' make the convergence better (also slower)

In [8]:
edg = pd.DataFrame(columns=columns)
from ngsolve.solvers import CG
val = 15 # bonusintorder
for order in orders:
    for size in mesh_size:
        eps_size = 0.0001*size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy], dgjumps = True)

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        ## Enrichment
        p = (coeff_x * px) + (coeff_y * py)
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        u_Other = us.Other() + coeff_x * px.Other() + coeff_y * py.Other()
        v_Other = vs.Other() + coeff_x * qx.Other() + coeff_y * qy.Other()
        
        jump_u = u - u_Other
        jump_v = v - v_Other
        
        n = specialcf.normal(2)
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        grad_uOther = grad(us.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py.Other() \
        + coeff_x * grad(px.Other()) + coeff_y * grad(py.Other())
        
        grad_vOther = grad(vs.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy.Other() \
        + coeff_x * grad(qx.Other()) + coeff_y * grad(qy.Other())
                
            
        mean_dudn = 0.5 * n * (grad_u + grad_uOther)
        mean_dvdn = 0.5 * n * (grad_v + grad_vOther)
        
        h = specialcf.mesh_size
        
        # diffusion
        diffusion = grad_u * grad_v * dx(bonus_intorder = val) \
        + alpha * order**2/h * jump_u * jump_v * dx(skeleton=True, bonus_intorder = val) \
        + (-mean_dudn* jump_v - mean_dvdn * jump_u) * dx(skeleton=True, bonus_intorder = val) \
        + alpha * order**2/h * u * v * ds(skeleton=True, bonus_intorder = val) \
        + (-n * grad_u * v - n * grad_v * u) * ds(skeleton=True, bonus_intorder = val)
                
        # convection
        b = CoefficientFunction((beta[0],beta[1]) )
        uup = IfPos(b*n, u, u_Other)
        convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * jump_v * dx(skeleton=True, bonus_intorder = val)
        convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * v * dx(element_boundary=True, bonus_intorder = val)
       
        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        acd.Assemble()

        # rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v, bonus_intorder = val)
        f.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = acd.mat.Inverse(inverse="umfpack") * f.vec
        #c = acd.mat.Inverse(inverse="sparsecholesky")
        #c = Preconditioner(acd,type="local",block=True,inverse="umfpack")
        #c.Update()
        #c.Test()
        #input("")
        #gfu.vec.data = CG(acd.mat,f.vec,c.mat)
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        edg.loc[len(edg)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')
        
#Draw (gfu.components[2] * coeff_y, mesh, 'test')
%store edg

Order: 1  Mesh Size: 1.0 L2 Error: 0.0011656810042903944
Order: 1  Mesh Size: 0.5 L2 Error: 0.0004612467303975414
Order: 1  Mesh Size: 0.25 L2 Error: 0.00012702830398321314
Order: 1  Mesh Size: 0.125 L2 Error: 4.287877934239961e-05
Order: 1  Mesh Size: 0.0625 L2 Error: 1.7600393399778967e-05
Order: 1  Mesh Size: 0.0313 L2 Error: 6.782273985758889e-06
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.00074159866999481
Order: 2  Mesh Size: 0.5 L2 Error: 0.00032150886607145465
Order: 2  Mesh Size: 0.25 L2 Error: 0.00013089718265863495
Order: 2  Mesh Size: 0.125 L2 Error: 4.454809870851653e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.2461083787423763e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 3.0899325375439e-06
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0006074460092310988
Order: 3  Mesh Size: 0.5 L2 Error: 0.00034400490512132714
Order: 3  Mesh Size: 0.25 L2 Error: 0.

#### Enrichment with HDG

In [4]:
ehdg = pd.DataFrame(columns=columns)
val = 15
#coeff_x = sin(x)
#coeff_y = cos(y)
for order in orders:
    for size in mesh_size:        
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        F = FacetFESpace(mesh, order=0, dirichlet="bottom|left|right|top")
        
        eps_size = 0.0001*size
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        
        gfF = GridFunction(F)
        
        gfF.vec[:] = 0
        for el in F.Elements():
            if ba_x[el.nr]:
                for dof in el.dofs:
                    gfF.vec[dof] += 1
        ba_F_x = BitArray(F.ndof)
        ba_F_x.Clear()
        for i in range(F.ndof):
            if gfF.vec[i] > 1.5:
                ba_F_x[i] = True

        gfF.vec[:] = 0
        for el in F.Elements():
            if ba_y[el.nr]:
                for dof in el.dofs:
                    gfF.vec[dof] += 1
        ba_F_y = BitArray(F.ndof)
        ba_F_y.Clear()
        for i in range(F.ndof):
            if gfF.vec[i] > 1.5:
                ba_F_y[i] = True
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        Fx = Compress(F, active_dofs=ba_F_x)
        Fy = Compress(F, active_dofs=ba_F_y)
        
        
        # Facets. Here the jumps aren't necessary. 
        #F = FacetFESpace(mesh, order=order)
        fes = FESpace([V, Qx, Qy, F, Fx, Fy])
        #print(fes.FreeDofs())

        (us, px, py, uhat, uhatx, uhaty), (vs, qx, qy, vhat, vhatx, vhaty) = fes.TnT()
        
        #coeff=exact
        p = (coeff_x * px) + (coeff_y * py) 
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        vhat = vhat + vhatx * coeff_x + vhaty * coeff_y
        uhat = uhat + uhatx * coeff_x + uhaty * coeff_y
        #vhat = vhat + vhatx * sin(x) + vhaty * sin(y)
        #uhat = uhat + uhatx * sin(x) + uhaty * sin(y)
        
        jump_u = u-uhat
        jump_v = v-vhat
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        #condense = True
        
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)
        dS = dx(element_boundary=True, bonus_intorder=val)
                
        diffusion = grad_u * grad_v *dx(bonus_intorder=val) + \
            alpha * order**2/h*jump_u*jump_v*dS + \
            (-grad_u *n*jump_v - grad_v *n*jump_u)*dS
#         diffusionVOL = SymbolicBFI(eps*grad_u * grad_v, bonus_intorder=15)
#         diffusionEB = SymbolicBFI(eps*alpha * order**2/h*jump_u*jump_v + \
#             (-eps*grad_u *n*jump_v - eps*grad_v *n*jump_u),element_boundary=True,bonus_intorder=15,simd_evaluate=False)
        
        
        b = CoefficientFunction((beta[0],beta[1]))
        uup = IfPos(b * n, u, uhat)
        convection = -b * u * grad_v *dx(bonus_intorder=val) + b * n * uup * jump_v * dS

        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        #acd += convection
        #acd += diffusionVOL
        #acd += diffusionEB
        acd.Assemble()
        #print(acd.mat)
        f = LinearForm(fes)
        f += SymbolicLFI(coeff*v,bonus_intorder=val)
        f.Assemble()

        gfu = GridFunction(fes)
        ## umfpack gives weird values
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky") * f.vec
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh, order=15))
        ehdg.loc[len(ehdg)] = [order, size, error] 
        Draw(u2,mesh,"u2")
        #input("")
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')

#Draw(u2, mesh, 'hdg')
%store ehdg    

Order: 1  Mesh Size: 1.0 L2 Error: 0.02383605930054911
Order: 1  Mesh Size: 0.5 L2 Error: 0.021374095870783084
Order: 1  Mesh Size: 0.25 L2 Error: 0.005118255713485026
Order: 1  Mesh Size: 0.125 L2 Error: 0.005559159307825272
Order: 1  Mesh Size: 0.0625 L2 Error: 0.0063705766161457865
Order: 1  Mesh Size: 0.0313 L2 Error: 0.005656297184402726
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.00844216816440501
Order: 2  Mesh Size: 0.5 L2 Error: 0.008521437616077713
Order: 2  Mesh Size: 0.25 L2 Error: 5.607603545486877
Order: 2  Mesh Size: 0.125 L2 Error: 0.005930934444529947
Order: 2  Mesh Size: 0.0625 L2 Error: 0.005840121548171368
Order: 2  Mesh Size: 0.0313 L2 Error: 0.0058871114231476935
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0093946553160595
Order: 3  Mesh Size: 0.5 L2 Error: 0.008895877051222103
Order: 3  Mesh Size: 0.25 L2 Error: 0.007348150315135115
Or

In [ ]:
Order: 1  Mesh Size: 1.0 L2 Error: 0.028388495093194802
Order: 1  Mesh Size: 0.5 L2 Error: 0.02571379811991953
Order: 1  Mesh Size: 0.25 L2 Error: 0.005904422734396558
Order: 1  Mesh Size: 0.125 L2 Error: 0.005404704164843337
Order: 1  Mesh Size: 0.0625 L2 Error: 0.005296168625879318
Order: 1  Mesh Size: 0.0313 L2 Error: 0.0053309775947820845
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.010496596293949176
Order: 2  Mesh Size: 0.5 L2 Error: 0.008728741331953643
Order: 2  Mesh Size: 0.25 L2 Error: 0.4611284976890837
Order: 2  Mesh Size: 0.125 L2 Error: 0.507603161665148
Order: 2  Mesh Size: 0.0625 L2 Error: 0.005608696604245726
Order: 2  Mesh Size: 0.0313 L2 Error: 0.005795270097610484
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.007377408955121438
Order: 3  Mesh Size: 0.5 L2 Error: 0.00950825959752038
Order: 3  Mesh Size: 0.25 L2 Error: 0.010878739030786382
Order: 3  Mesh Size: 0.125 L2 Error: 0.023907253675992394
Order: 3  Mesh Size: 0.0625 L2 Error: 0.008833036208798021
Order: 3  Mesh Size: 0.0313 L2 Error: 0.009371516463999415